# Reformat catalog for CDS upload
Setup catalog info below

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_dir_name = "./vasca_pipeline/"+region_name+"/"
region_cat_fname = region_dir_name+"region_"+region_name+"_cat.fits"
print_latex = True
tabnames_4CDS = ["tt_sources","tt_detections","tt_fields","tt_visits", "tt_coadd_sources"] #,"tt_filters"

cat_cols = ['src_name','nr_det', 'ra', 'dec', 'pos_err', 'pos_xv', 'pos_var', 'pos_cpval', 'pos_rchiq',
            'flux', 'flux_err', 'flux_nxv', 'flux_var', 'flux_cpval', 'flux_rchiq',"coadd_src_id", 'coadd_ffactor', 'coadd_fdiff_s2n',
            'rg_src_id', 'nr_fd_srcs', 'hr', 'hr_err',"ogrp_simbad","otype_simbad","main_id_simbad","Source_gaiadr3","WDJname_gaiaedr3_wd","objid_gfcat","ls_peak_pval", "ls_peak_freq"]

In [2]:
%matplotlib widget
import os
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.stats import chi2

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.table import Table,Column
import numpy as np
from matplotlib.ticker import ScalarFormatter

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, flux2mag, select_obs_filter,freq2period,period2freq,flux2mag_np,mag2flux_np, dd_filter2id,get_flat_table
from vasca.tables_dict import dd_vasca_columns
import cdspyreadme

rc = Region()
rc.load_from_fits(region_cat_fname)

In [3]:
###Copy info from other tzables into tt_sources

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp", "main_id"],"simbad_match_id")
#rc.tt_sources.rename_column("match_distance","match_distance_simbad"),"match_distance"
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
rc.tt_sources.rename_column("otype","otype_simbad")
rc.tt_sources.rename_column("main_id","main_id_simbad")

copy_cols = ["Source"] #,,"Plx_dist","AG","RPlx", ,"AG",,"RPlx",,"Plx"   ,"BP-RP","Gmag_abs"
rc.copy_table_columns("tt_sources","tt_gaiadr3",copy_cols,"gaiadr3_match_id")
for col in copy_cols:
    rc.tt_sources.rename_column(col,col+"_gaiadr3")

rc.copy_table_columns("tt_sources","tt_gaiaedr3_wd",["WDJname"],"gaiaedr3_wd_match_id") # ,"Pwd"
rc.tt_sources.rename_column("WDJname","WDJname_gaiaedr3_wd")
#rc.tt_sources.rename_column("Pwd","Pwd_gaiaedr3_wd")

rc.copy_table_columns("tt_sources","tt_gfcat",["gfcat_objid"],"gfcat_src_id")
rc.tt_sources.rename_column("gfcat_objid","objid_gfcat")

rc.tt_sources.remove_columns(["gaiaedr3_wd_match_id","simbad_match_id","gaiadr3_match_id","gfcat_src_id"]) 

rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_peak_freq"],"rg_src_id")


### Modify tables

# Add minimum position error of 0.4 arc sec for systematics
rc.tt_sources.remove_columns(["flux_ne"])
rc.tt_sources["pos_err"][rc.tt_sources["pos_err"]<0.4]=0.4
rc.tt_sources = rc.tt_sources[cat_cols+["sel","obs_filter_id"]]

#display(rc.tt_sources)

# Add filter to tt_detections
rc.add_column("tt_detections", "obs_filter", col_data=None)
rc.tt_detections["obs_filter"][rc.tt_detections["obs_filter_id"]==1]="NUV"
rc.tt_detections["obs_filter"][rc.tt_detections["obs_filter_id"]==2]="FUV"
rc.tt_detections.remove_column("obs_filter_id")

#Remove columns from coadd sources
rm_coadd_cols = ["pos_xv","pos_var","pos_cpval","pos_rchiq","coadd_dist","flux_nxv","flux_var","flux_cpval","flux_rchiq","coadd_ffactor","coadd_fdiff_s2n","rg_fd_id","nr_fd_dets"] #,,"rg_src_id"
rc.tt_coadd_sources.remove_columns(rm_coadd_cols)
rc.tt_coadd_sources["pos_err"][rc.tt_coadd_sources["pos_err"]<0.4]=0.4
display(rc.tt_coadd_sources)

#Remove unused field columns
#rc.tt_fields.remove_columns(["observatory",])

#Change description for filter ID in visits
rc.tt_visits["obs_filter_id"].description = "Observation filter, 1=NUV, 2=FUV, 3=both"

fd_src_id,nr_det,ra,dec,pos_err,coadd_src_id,obs_filter_id,sel,flux,flux_err,rg_src_id
,,deg,deg,arcsec,,,,1e-06 Jy,1e-06 Jy,
int32,int32[2],float64,float64,float32,int64,int32[2],bool,float32[2],float32[2],int32
-1,3 .. 2,53.01019686171658,-27.27185778662949,0.4,61,1 .. 2,True,147.36476 .. 82.93363,0.21306285 .. 0.47076347,3962
-1,2 .. 2,306.5087473500289,-43.96201083529074,0.4,108,1 .. 2,True,90.95539 .. 37.56389,0.33069217 .. 0.45907465,10430
-1,2 .. 2,244.9729381664936,55.12462784698586,0.4,146,1 .. 2,True,19.705061 .. 6.6761193,0.13458285 .. 0.30325738,11280
-1,2 .. 2,244.95163752211394,55.13903652728861,0.4,149,1 .. 2,True,456.61932 .. 72.96044,0.53369516 .. 0.8878632,11291
-1,2 .. 2,244.94605897958323,55.39501812862455,0.4,150,1 .. 2,True,6.9047728 .. 3.4646213,0.099998936 .. 0.23654515,11292
-1,2 .. 2,244.9143923278137,55.41736331131758,0.4,152,1 .. 2,True,7.65115 .. 3.5899456,0.10207792 .. 0.23550881,11305
...,...,...,...,...,...,...,...,...,...,...
-1,1 .. -1,4.881798693731509,16.31148548571217,0.47983882,1264741,1 .. 0,True,28.080936 .. -1.0,0.14780109 .. -1.0,391763


### Print out LATEX for paper tables

In [4]:
#Latex table names print out and tt_sources  columns
if print_latex:
    for tabname in tabnames_4CDS:
        new_tab_name = tabname.replace("tt_","").upper()
        print(new_tab_name.replace("_","\_"), "&  \\\\")
    
    print("\n")
    for col in cat_cols:
        icol = dd_vasca_columns[col.replace("_simbad","").replace("_gaiadr3","").replace("_gaiaedr3_wd","").replace("objid_gfcat","gfcat_objid")]
        print(col.replace("_","\_").upper(),"&",icol["description"],"&",icol["unit"].replace("1",""), "\\\\")

SOURCES &  \\
DETECTIONS &  \\
FIELDS &  \\
VISITS &  \\
COADD\_SOURCES &  \\


SRC\_NAME & VASCA catalog source name &  \\
NR\_DET & Number of detections &  \\
RA & Sky coordinate Right Ascension (J2000) & degree \\
DEC & Sky coordinate Declination (J2000) & degree \\
POS\_ERR & Sky coordinate position error & arcsec \\
POS\_XV & Sky position excess variance & arcsec2 \\
POS\_VAR & Sky position variance & arcsec2 \\
POS\_CPVAL & Sky position quality &  \\
POS\_RCHIQ & Sky position reduced chisquared of the constant mean &  \\
FLUX & Flux density & e-6Jy \\
FLUX\_ERR & Flux density error & e-6Jy \\
FLUX\_NXV & Flux normalized excess variance &  \\
FLUX\_VAR & Flux variance & e-2Jy2 \\
FLUX\_CPVAL & Probability value for a constant flux from the chisquare test &  \\
FLUX\_RCHIQ & Flux reduced chisquared of the constant mean &  \\
COADD\_SRC\_ID & Co-add source ID number &  \\
COADD\_FFACTOR & Source flux divided by flux of the associated co-add source &  \\
COADD\_FDIFF\_S2N & Signal to

### Rename table columns, remove unwanted columns and store into CDS table

In [5]:
#print(rc.tt_sources.colnames)
tablemaker = cdspyreadme.CDSTablesMaker()


for tabname in tabnames_4CDS:
    print(tabname)
    tt = get_flat_table(rc.__dict__[tabname])
    for colname in tt.colnames:
        new_col_name = colname.upper()
        if new_col_name not in tt.colnames:
            tt.rename_column(colname,new_col_name)


        #tt.rename_column("MAINCAT_MATCH_ID","WDMAIN_MATCH_ID")

# rc.tt_sources.remove_columns(["GFCAT_DIST","FD_SRC_ID","COADD_DIST","SEL","RG_FD_ID"])
    if "RA" in tt.colnames:
        if tabname == "tt_gaiadr3":
            tt.rename_column("RAJ2000","RA_GAIA")
            tt.rename_column("DEJ2000","DEC_GAIA")
            
        tt.rename_column("RA","RAJ2000")
        tt.rename_column("DEC","DECJ2000")
    remove_cols = ["GFCAT_DIST","COADD_DIST","SEL","RG_FD_ID","OBS_FILTER_ID_NUV","OBS_FILTER_ID_FUV"]
    if tabname not in ["tt_src_id_map", "tt_detections", "tt_coadd_detections"]:
        remove_cols.append("FD_SRC_ID")
    for rcol in remove_cols:
        if rcol in tt.colnames:
            tt.remove_column(rcol) 
    display(tt[0:2])

    tablemaker.addTable(tt, name=tabname.replace("tt_","").upper())

tt_sources


SRC_NAME,RAJ2000,DECJ2000,POS_ERR,POS_XV,POS_VAR,POS_CPVAL,POS_RCHIQ,COADD_SRC_ID,RG_SRC_ID,NR_FD_SRCS,HR,HR_ERR,OGRP_SIMBAD,OTYPE_SIMBAD,MAIN_ID_SIMBAD,SOURCE_GAIADR3,WDJNAME_GAIAEDR3_WD,OBJID_GFCAT,LS_PEAK_PVAL,LS_PEAK_FREQ,NR_DET_NUV,NR_DET_FUV,FLUX_NUV,FLUX_FUV,FLUX_ERR_NUV,FLUX_ERR_FUV,FLUX_NXV_NUV,FLUX_NXV_FUV,FLUX_VAR_NUV,FLUX_VAR_FUV,FLUX_CPVAL_NUV,FLUX_CPVAL_FUV,FLUX_RCHIQ_NUV,FLUX_RCHIQ_FUV,COADD_FFACTOR_NUV,COADD_FFACTOR_FUV,COADD_FDIFF_S2N_NUV,COADD_FDIFF_S2N_FUV
,deg,deg,arcsec,arcsec2,arcsec2,,,,,,,,,,,,,,,1 / d,,,1e-06 Jy,1e-06 Jy,1e-06 Jy,1e-06 Jy,,,1e-12 Jy2,1e-12 Jy2,,,,,,,,
bytes24,float64,float64,float32,float32,float32,float32,float32,int64,int32,int32,float32,float32,bytes8,bytes32,bytes32,bytes32,bytes32,bytes32,float32,float32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1UVA J100046.8+20404.7,150.19490602211764,2.0679693223742093,0.4,-0.3145582,0.1782018,1.0,0.3056839,846948,41,10,0.5638935,0.017211191,AGN,Sy1,2XMM J100046.7+020404,3836069006998531584,none,none,0.0015057425,0.9750193,47,23,13.532883,7.885985,0.12551127,0.21120642,0.016849143,0.049125932,4.0856714,4.8104725,1.2275191e-28,5.1074105e-15,5.287477,5.337544,0.87979734,-100.0,-10.652894,-10000.0
1UVA J100035.0+20235.2,150.14575525319893,2.043119350119519,0.4,-0.4227665,0.21190573,0.9999997,0.35446438,60853,68,10,0.31352478,0.016040279,AGN,QSO,[VV2006] J100034.9+020235,3836046123412862208,none,none,8.094213e-05,0.9984624,52,21,9.605367,3.214509,0.110952616,0.14565498,0.03246673,0.124309875,3.953956,1.8266395,4.6e-44,1.6166583e-10,6.637378,4.4029164,0.9722032,-100.0,-1.9600928,-10000.0


tt_detections


VIS_ID,FD_SRC_ID,RAJ2000,DECJ2000,POS_ERR,FLUX,FLUX_ERR,FLUX_APP_RATIO,S2N,R_FOV,ARTIFACTS,CLASS_STAR,CHKOBJ_TYPE,SIZE_WORLD,ELLIP_WORLD,FLUX_AUTO,FLUX_AUTO_ERR,RG_SRC_ID,OBS_FILTER
,,deg,deg,arcsec,1e-06 Jy,1e-06 Jy,,,deg,,,,arcsec,,Jy,Jy,,
uint64,int32,float64,float64,float32,float32,float32,float32,float32,float32,int64,float32,int32,float32,float32,float32,float32,int32,bytes8
2518745845218672640,200,53.413730851927276,-28.289945007098186,0.62011015,27.90702,1.1415521,0.70133233,21.760221,0.4971887,1,0.97624475,0,3.9381106,0.22328553,30.128954,1.3845885,22515,NUV
2518745845218672640,624,53.30202025247929,-28.194783952584167,0.6614574,8.241435,0.7498808,0.87157047,9.216578,0.36643726,0,0.9832993,0,2.5385818,0.16107683,7.507178,0.8145299,65533,NUV


tt_fields


FIELD_ID,FIELD_NAME,PROJECT,RAJ2000,DECJ2000,OBSERVATORY,OBS_FILTER,FOV_DIAM,NR_VIS,TIME_BIN_SIZE_SUM,TIME_START,TIME_STOP
,,,deg,deg,,,deg,,s,d,d
bytes32,bytes32,bytes32,float64,float64,bytes22,bytes8,float32,int32,float32,float64,float64
GNU2509140647668088832,PKS2155m304,DIS,329.7262466081879,-30.2657589090371,GALEX,NUV,1.2,1,240.05,54691.15467592593,54931.20467897769
GNU2509211016412266496,HPQ_3C454p3,DIS,343.48735799747396,16.1446550539213,GALEX,NUV,1.2,1,103.0,54738.14524305556,54841.14524305556


tt_visits


VIS_ID,TIME_BIN_START,TIME_BIN_SIZE,OBS_FILTER_ID
,d,s,
uint64,float64,float32,int32
2482083728193486848,52925.31974537037,1640.05,3
2482083728227041280,52925.593773148146,1641.1,3


tt_coadd_sources


RAJ2000,DECJ2000,POS_ERR,COADD_SRC_ID,RG_SRC_ID,NR_DET_NUV,NR_DET_FUV,FLUX_NUV,FLUX_FUV,FLUX_ERR_NUV,FLUX_ERR_FUV
deg,deg,arcsec,,,,,1e-06 Jy,1e-06 Jy,1e-06 Jy,1e-06 Jy
float64,float64,float32,int64,int32,int32,int32,float32,float32,float32,float32
53.01019686171658,-27.27185778662949,0.4,61,3962,3,2,147.36476,82.93363,0.21306285,0.47076347
306.5087473500289,-43.96201083529074,0.4,108,10430,2,2,90.95539,37.56389,0.33069217,0.45907465


In [6]:
os.chdir(region_dir_name+"/CDS")
tablemaker.writeCDSTables()


tablemaker.title = "The time-variable ultraviolet sky: Active Galactic Nuclei, Stars and White Dwarfs"
tablemaker.author = 'R. Buehler and J. Schliwinski'
tablemaker.date = 2024
tablemaker.abstract = """Here, we present the 1UVA catalog of time variable ultraviolet (UV) sources. We describe a new analysis pipeline, the VAriable Source
Clustering Analysis (VASCA). We apply the pipeline to 10 years of data from the GALaxy Evolution eXplorer (GALEX) satellite.
We analyse a sky area of 302 deg2 , resulting in the detection of 4202 time-variable UV sources. We cross correlate these sources with
multi-frequency data from the Gaia satellite and the SIMBAD database, finding an association for 3655 sources. The source sample
is dominated by Active Galactic Nuclei (≈73 %) and stars (≈24 %). We look at UV and multi-frequency properties of these sources,
focusing on the stellar population. We find UV variability for four White Dwarfs. One of them, WD J004917.14-252556.81, has
recently been found to be the most massive pulsating White Dwarf. Its Spectral Energy Distribution shows no sign of a stellar
companion. The observed flux variability is unexpected and difficult to explain.."""
#tablemaker.more_description = """The catalog paper has the title "The time-variable ultraviolet sky: Active Galactic Nuclei, Stars and White Dwarfs" """
tablemaker.keywords = "Ultraviolet: general, stars, galaxies – Galaxies: active – Stars: Hertzsprung-Russell and C-M diagrams, white dwarfs"

with open("ReadMe", "w") as fd:
    tablemaker.makeReadMe(out=fd)

In [7]:
display(rc.tt_simbad[0:2])
display(rc.tt_gaiadr3[0:2])
display(rc.tt_gfcat[0:2])
display(rc.tt_gaiaedr3_wd[0:2])
display(rc.tt_sources[0:2])
display(rc.tt_lombscargle[0:2])

main_id,ra,dec,ra_prec,dec_prec,coo_err_maja,coo_err_mina,coo_err_angle,coo_qual,coo_wavelength,coo_bibcode,otype,otypes,distance_distance,distance_q,distance_unit,distance_merr,distance_perr,distance_method,distance_bibcode,match_distance,rvz_type,rvz_radvel,rvz_error,rvz_qual,rvz_wavelength,rvz_bibcode,z_value,sp_type,sp_qual,sp_bibcode,script_number_id,rg_src_id,simbad_match_id,ogrp
,deg,deg,,,mas,mas,deg,,,,,,,,,,,,,arcsec,,km / s,km / s,,,,,,,,,,,
bytes32,float64,float64,int16,int16,float32,float32,int16,bytes1,bytes1,bytes32,bytes32,bytes32,float64,bytes1,bytes4,float64,float64,bytes8,bytes19,float32,bytes1,float64,float32,bytes1,bytes1,bytes32,float64,bytes32,bytes32,bytes32,int32,int32,int32,bytes8
2XMM J100046.7+020404,150.19472291666665,2.067906388888889,14,14,2.287,3.314,90,C,O,2020yCat.1350....0G,Sy1,Sy1|AGN|AGN|AGN|AGN|AGN|*|AG?|G|,--,--,--,--,--,--,--,0.6957,z,0.552,0.000,C,O,2009ApJ...696.1195T,0.5524000,--,--,--,1,41,0,AGN
[VV2006] J100034.9+020235,150.14558874999997,2.043054722222222,14,14,1.578,2.045,90,C,O,2020yCat.1350....0G,QSO,QSO|AGN|QSO|AGN|AGN|AGN|AGN|AGN|,--,--,--,--,--,--,--,0.6421,z,1.177,0.001,C,O,2009ApJ...696.1195T,1.1767000,--,--,--,2,68,1,AGN


_q,ra,dec,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,PQSO,PGal,PSS,RPlx,VarFlag,o_Gmag,RFRP,RFBP,AG,E_BP-RP,rg_src_id,match_distance,gaiadr3_match_id,Gmag_abs,Plx_dist,ogrp
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,,,,,,,mag,mag,,arcsec,,,pc,
int32,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,bytes12,bytes15,bytes19,int32,bytes10,bytes16,bytes17,bytes21,float64,float64,float64,float64,float64,float64,bytes13,int16,float32,float32,float32,float32,int32,float64,int32,float32,float64,bytes8
1,150.2,2.1,3836069006998531584,2.2872,3.3137,--,--,--,--,--,--,--,--,95.82888,1.689,20.733625,93.75,16.38,20.408556,130.9,9.985,19.455320,0.953236,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110481501947501974,1237651753997304055,--,--,--,J100046.74+020404.4,--,N6SS000981,--,--,--,150.2,2.1,0.001869,0.000724,0.996765,--,NOT_AVAILABLE,79,13.11,5.725,--,--,41,0.6956587221689177,0,-100.0,--,none
2,150.1,2.0,3836046123412862208,1.5776,2.0453,--,--,--,--,--,--,--,--,90.12499,2.676,20.800253,37.12,20.52,21.414429,59.56,10.83,20.310590,1.103838,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110451501456232172,1237651736280760581,--,--,--,J100034.94+020234.9,--,N6SS004829,--,--,--,150.1,2.0,0.001265,0.000013,0.997987,--,NOT_AVAILABLE,43,5.497,1.809,--,--,68,0.6420858569707824,1,-100.0,--,none


gfcat_objid,ra,dec,n_gfcat,simbad_id,simbad_otype,simbad_match_offset,simbad_n_match,n_galex,gfcat_src_id,rg_src_id
,deg,deg,,,,,,,,
bytes24,float64,float64,int64,bytes44,bytes52,bytes9,int64,int64,int32,int32
GFCAT J163938.3+410644.4,249.909726,41.112356,9,V* AF Her,RRLyrae,2.021917,1,84,0,-1
GFCAT J204615.6-045550.4,311.565113,-4.93067,1,2MASS J20461427-0456025,Low-Mass*,23.589757,0,13,1,-1


_q,WDJname,GaiaEDR3,ra,dec,Plx,Pwd,pmRA,pmDE,Gmag,BPmag,RPmag,TeffH,loggH,MassH,TeffHe,loggHe,MassHe,rgeo,rpgeo,SDSS12,umag,gmag,rmag,imag,zmag,_RA.icrs,_DE.icrs,meanAV,minAV,maxAV,rg_src_id,match_distance,gaiaedr3_wd_match_id
,,,deg,deg,mas,,mas / yr,mas / yr,mag,mag,mag,K,log(cm.s**-2),solMass,K,log(cm.s**-2),solMass,pc,pc,,mag,mag,mag,mag,mag,deg,deg,mag,mag,mag,,arcsec,
int32,bytes23,int64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,bytes19,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,int32,float64,int32
197,WDJ202622.19-434031.74,6679051075906933888,306.6,-43.7,0.6442,0.520516,11.036,-3.767,19.9644,20.0073,19.8234,--,--,--,--,--,--,1718.51770000,2819.34839000,--,--,--,--,--,--,306.6,-43.7,0.130,0.130,0.130,10407,0.29353535062614117,0
315,WDJ104356.75+580731.92,855167540988615296,161.0,58.1,5.4931,0.144880,-61.045,-19.888,17.2745,17.4169,16.8548,--,--,--,--,--,--,181.05345200,182.06933600,J104356.73+580731.8,17.765,17.537,17.635,17.623,17.189,161.0,58.1,0.030,0.030,0.030,17756,0.8107444132924932,1


src_name,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_src_id,coadd_ffactor,coadd_fdiff_s2n,rg_src_id,nr_fd_srcs,hr,hr_err,ogrp_simbad,otype_simbad,main_id_simbad,Source_gaiadr3,WDJname_gaiaedr3_wd,objid_gfcat,ls_peak_pval,ls_peak_freq,sel,obs_filter_id
,,deg,deg,arcsec,arcsec2,arcsec2,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,,,,,,,1 / d,,
bytes24,int32[2],float64,float64,float32,float32,float32,float32,float32,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int64,float32[2],float32[2],int32,int32,float32,float32,bytes8,bytes32,bytes32,bytes32,bytes32,bytes32,float32,float32,bool,int32[2]
1UVA J100046.8+20404.7,47 .. 23,150.19490602211764,2.0679693223742093,0.4,-0.3145582,0.1782018,1.0,0.3056839,13.532883 .. 7.885985,0.12551127 .. 0.21120642,0.016849143 .. 0.049125932,4.0856714 .. 4.8104725,1.2275191e-28 .. 5.1074105e-15,5.287477 .. 5.337544,846948,0.87979734 .. -100.0,-10.652894 .. -10000.0,41,10,0.5638935,0.017211191,AGN,Sy1,2XMM J100046.7+020404,3836069006998531584,none,none,0.0015057425,0.9750193,True,1 .. 2
1UVA J100035.0+20235.2,52 .. 21,150.14575525319893,2.043119350119519,0.4,-0.4227665,0.21190573,0.9999997,0.35446438,9.605367 .. 3.214509,0.110952616 .. 0.14565498,0.03246673 .. 0.124309875,3.953956 .. 1.8266395,4.6e-44 .. 1.6166583e-10,6.637378 .. 4.4029164,60853,0.9722032 .. -100.0,-1.9600928 .. -10000.0,68,10,0.31352478,0.016040279,AGN,QSO,[VV2006] J100034.9+020235,3836046123412862208,none,none,8.094213e-05,0.9984624,True,1 .. 2


rg_src_id,ls_peak_power,ls_peak_freq,ls_peak_pval,ls_pval_alt_flt,ls_model_rchiq,ls_model_pval
,,1 / d,,,,
int32,float32,float32,float32,float32,float32,float32
41,0.44715345,0.9750193,0.0015057425,0.9999908,3.068476,3.5325888e-11
68,0.4836102,0.9984624,8.094213e-05,0.12976168,3.5829577,3.7953617e-16
